In [2]:
# Load Image
import numpy as np, os 
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [3]:
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
def get_bbox(img_arr):
    detector_output = model(img_arr)
    class_prob,class_id, class_boxes = detector_output["detection_scores"], detector_output["detection_classes"], detector_output["detection_boxes"]

    prob_max=tf.argmax(class_prob, axis=-1).numpy()[0]
    class_label=class_id.numpy()[0][prob_max]
    class_box=class_boxes.numpy()[0][prob_max]
    return prob_max, class_box

In [35]:
# Load images and apply patch
img_path='/content/images/'
patch_path = '/content/patch.jpg'

for path in os.listdir(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path+path, grayscale=False, target_size=(512,512))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = np.array([img_arr]) 

    prob_max, class_box=get_bbox(img_arr)

    p_size=int((class_box[3]-class_box[1])*512//2.5)
    patch = tf.keras.preprocessing.image.load_img(patch_path, grayscale=False, target_size=(p_size,p_size))
    patch_arr = tf.keras.preprocessing.image.img_to_array(patch)
    patch_arr = np.array([patch_arr]) 

    p_y=int((class_box[2]-class_box[0])*512//2)
    p_x=int((class_box[3]-class_box[1])*512//2)
    img_arr[0][p_y:p_y+p_size,p_x:p_x+p_size]=patch_arr

    from keras.preprocessing.image import save_img
    save_img(patched_img_path+path, img_arr[0])

In [ ]:
  # Calculate gradient
  loss_object = tf.keras.losses.CategoricalCrossentropy()
  original_label=tf.constant([0.85,0.40]) # We will train till People score is 0.40

def calc_gradient(img):
    with tf.GradientTape() as gt:
        # Define the calculation that needs to be derived
        gt.watch(img_tf)
        img_tf=tf.cast(img_tf, tf.uint8)
        detector_output = model(img_tf)
        cls_outputs = detector_output["detection_scores"]
        prediction=cls_outputs[0][0:2]
        loss = loss_object(original_label, prediction)
    
    # Get the gradients for the loss w.r.t image
    grads = gt.gradient(loss, img_tf) 


In [ ]:
# train to update patch
patched_img_path='/content/patched_images/'
!mkdir {patched_img_path}

for path in os.listdir(patched_img_path):

    img = tf.keras.preprocessing.image.load_img(img_path+path, grayscale=False, target_size=(512,512))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = np.array([img_arr]) 
    grad=calc_gradient(img)

    #Get class and BBox
    prob_max, class_box=get_bbox(img_arr)

    p_size=int((class_box[3]-class_box[1])*512//2.5)
    p_y=int((class_box[2]-class_box[0])*512//2)
    p_x=int((class_box[3]-class_box[1])*512//2)

    img_arr[0][p_y:p_y+p_size,p_x:p_x+p_size]=patch_arr
    patch=img_arr[0][p_y:p_y+p_size,p_x:p_x+p_size]
    
    patch=patch - eps*grad
    img_arr[0][p_y:p_y+p_size,p_x:p_x+p_size]=patch

    from keras.preprocessing.image import save_img
    save_img(patched_img_path+path, img_arr[0])

In [5]:
# Calculate accuracy

img_class=[]
for path in os.listdir(img_path):

    img = tf.keras.preprocessing.image.load_img(img_path+path, grayscale=False, target_size=(512,512))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = np.array([img_arr]) 
    prob_max, class_box=get_bbox(img_arr)
    img_class.append(np.argmax(prob_max))

acc=sum(img_class!=0)/len(img_class)

print(f'accuracy - {acc}')

accuracy - 0.957


In [6]:
# Test accuracy
test_img_path='/content/val_images/'
test_img_class=[]
for path in os.listdir(test_img_path):

    img = tf.keras.preprocessing.image.load_img(img_path+path, grayscale=False, target_size=(512,512))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = np.array([img_arr]) 

    prob_max, class_box=get_bbox(img_arr)

    p_size=int((class_box[3]-class_box[1])*512//2.5)
    p_y=int((class_box[2]-class_box[0])*512//2)
    p_x=int((class_box[3]-class_box[1])*512//2)

    img_arr[0][p_y:p_y+p_size,p_x:p_x+p_size]=patch # apply trained patch
    # Check class again
    prob_max, class_box=get_bbox(img_arr)
    test_img_class.append(np.argmax(prob_max))

acc=sum(test_img_class!=0)/len(test_img_class)

print(f'accuracy - {acc}')

accuracy - 0.89
